## Installing Dependencies

In [7]:
import os
# !git clone https://github.com/Open-Speech-EkStep/vakyansh-tts
# os.chdir('vakyansh-tts') 
# !bash install.sh
# !python setup.py bdist_wheel
# !pip install -e .
# !pip install unidecode pydload
# os.chdir('tts_infer')
# !wget -q https://storage.googleapis.com/vakyansh-open-models/translit_models.zip
# !unzip translit_models.zip
# !rm translit_models.zip
# os.chdir('translit_models/hindi')

# !wget https://storage.googleapis.com/vakyansh-open-models/tts/hindi/hi-IN/female_voice_0/glow.zip
# !unzip glow.zip

# !wget https://storage.googleapis.com/vakyansh-open-models/tts/hindi/hi-IN/female_voice_0/hifi.zip
# !unzip hifi.zip

# !rm glow.zip
# !rm hifi.zip

# os.chdir('/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts')

## Inference Code

In [8]:
# !unzip glow.zip

# !wget https://storage.googleapis.com/vakyansh-open-models/tts/hindi/hi-IN/female_voice_0/hifi.zip
# !unzip hifi.zip

# !rm glow.zip
# !rm hifi.zip

os.chdir('/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts')

In [9]:
from tts_infer.tts import TextToMel, MelToWav
from tts_infer.transliterate import XlitEngine
from tts_infer.num_to_word_on_sent import normalize_nums

import re
from scipy.io.wavfile import write
device = 'cpu'

text_to_mel = TextToMel(glow_model_dir='/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/translit_models/hindi/glow_ckp', device=device)
mel_to_wav = MelToWav(hifi_model_dir='/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/translit_models/hindi/hifi_ckp', device=device)

def translit(text, lang):
    reg = re.compile(r'[a-zA-Z]')
    engine = XlitEngine(lang)
    words = [engine.translit_word(word, topk=1)[lang][0] if reg.match(word) else word for word in text.split()]
    updated_sent = ' '.join(words)
    return updated_sent
    
def run_tts(text, lang):
    text = text.replace('।', '.') # only for hindi models
    text_num_to_word = normalize_nums(text, lang) # converting numbers to words in lang
    text_num_to_word_and_transliterated = translit(text_num_to_word, lang) # transliterating english words to lang
    
    mel = text_to_mel.generate_mel(text_num_to_word_and_transliterated)
    audio, sr = mel_to_wav.generate_wav(mel)
    write(filename='temp.wav', rate=sr, data=audio) # for saving wav file, if needed
    return (sr, audio)

/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/translit_models/hindi/glow_ckp/G_250.pth
INFO:root:Loaded checkpoint '/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/translit_models/hindi/glow_ckp/G_250.pth' (iteration 250)


/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/../src/glow_tts/modules.py:234: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2355.)
  w_init = torch.qr(torch.FloatTensor(self.n_split, self.n_split).normal_())[0]


/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/translit_models/hindi/hifi_ckp/g_00100000
Loading '/Users/badri/Desktop/sem2/CS772/project/notebooks/tts/vakyansh-tts/tts_infer/translit_models/hindi/hifi_ckp/g_00100000'
Complete.
Removing weight norm...


In [3]:
_, audio = run_tts('गुलाबी कार्ड पोनी छोटी लड़कियों के लिए नॉक-जीन बेचती है। पिंजरे में बबूआ आपको बताता है कि आप बड़े होने पर क्या होंगे।', 'hi')

Loading hi...


## Results

In [4]:
import IPython.display as ipd
ipd.Audio('temp.wav')